<a href="https://colab.research.google.com/github/tantle27/tantle27/blob/main/Final_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Imports and Setup


In [84]:
"""
Efficient ANN Search with SOAR (Orthogonal Residuals) and FAISS

This script implements the following key components:
1. Data Loading and Normalization (from an HDF5 dataset)
2. PCA Transformation on training and testing data
3. Partitioning (Clustering) of data using MiniBatchKMeans
4. Definition of the SOAR Model with multiple residual encoders
5. Training of the SOAR Model with a combined MSE and Orthogonality Loss
6. Building a FAISS HNSW index on the encoded representations
7. Dynamic Reranking and Recall Evaluation for ANN retrieval
8. Hyperparameter Tuning Functions for clustering and lambda (orthogonality strength)

Required installations:
    !pip install faiss-cpu
    !pip install scikit-learn
    !pip install h5py
    !pip install pca  # if needed; check if PCA from scikit-learn suffices
"""


!pip install scikit-learn
!pip install h5py
!pip install pca
!pip install faiss-cpu
!wget --continue http://ann-benchmarks.com/sift-128-euclidean.hdf5

import time
import psutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import faiss
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
import h5py


--2025-04-13 20:33:12--  http://ann-benchmarks.com/sift-128-euclidean.hdf5
Resolving ann-benchmarks.com (ann-benchmarks.com)... 104.21.96.1, 104.21.112.1, 104.21.32.1, ...
Connecting to ann-benchmarks.com (ann-benchmarks.com)|104.21.96.1|:80... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



# Section 2: Data Loading and Preprocessing


In [85]:
def load_sift_hdf5(file_path, dataset_name='data'):
    """Load data from an HDF5 file."""
    with h5py.File(file_path, 'r') as f:
        data = np.array(f[dataset_name])
    return data

def load_and_normalize_data_hdf5(file_path, dataset_name='data'):
    """Load data and normalize it to unit length."""
    data = load_sift_hdf5(file_path, dataset_name)
    norms = np.linalg.norm(data, axis=1, keepdims=True)
    return data / (norms + 1e-8)

def apply_pca(train_data, test_data, n_components=96):
    """Apply PCA transformation to reduce dimensionality."""
    pca = PCA(n_components=n_components)
    pca.fit(train_data)
    train_data_pca = pca.transform(train_data)
    test_data_pca = pca.transform(test_data)
    return train_data_pca.astype('float32'), test_data_pca.astype('float32')


# Section 3: Data Partitioning and Residual Computation


In [86]:
def partition_data(data, n_clusters=256, random_state=42):
    """
    Partition data using MiniBatchKMeans clustering.
    Returns cluster labels and cluster centers.
    """
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1024)
    labels = kmeans.fit_predict(data)
    centers = kmeans.cluster_centers_.astype('float32')
    return labels, centers



# Section 4: SOAR Model Definition (Residual Encoders)


In [87]:

class ResidualEncoder(nn.Module):
    """
    A simple fully connected layer to transform residual vectors.
    The output is L2 normalized.
    """
    def __init__(self, input_dim, rep_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, rep_dim)

    def forward(self, x):
        rep = self.fc(x)
        rep = rep / (rep.norm(dim=1, keepdim=True) + 1e-8)
        return rep

class SOARModel(nn.Module):
    """
    The SOAR model uses multiple ResidualEncoder blocks to produce
    orthogonal components. The outputs are aggregated to form the final representation.
    """
    def __init__(self, input_dim, rep_dim, num_reps=4):
        super().__init__()
        self.encoders = nn.ModuleList([ResidualEncoder(input_dim, rep_dim) for _ in range(num_reps)])

    def forward(self, x):
        reps = [encoder(x) for encoder in self.encoders]
        # Sum all the representations to produce the combined representation
        combined_rep = torch.stack(reps, dim=0).sum(dim=0)
        return combined_rep, reps

def orthogonality_loss(reps):
    """
    Compute the orthogonality loss as the mean absolute dot product
    between each distinct pair of encoder outputs.
    """
    loss = 0.0
    num_reps = len(reps)
    for i in range(num_reps):
        for j in range(i + 1, num_reps):
            dot_product = (reps[i] * reps[j]).sum(dim=1)
            loss += torch.mean(torch.abs(dot_product))
    return loss

def soar_loss(target, pred, reps, lam=0.1):
    """
    Combined loss: Mean Squared Error between the aggregated prediction
    and the target, plus a weighted orthogonality loss.
    """
    mse = nn.MSELoss()(pred, target)
    ortho = orthogonality_loss(reps)
    return mse + lam * ortho


# Section 5: Training Function for the SOAR Model


In [88]:
def train_soar_model(data, labels, centers, input_dim, rep_dim, num_reps=4, epochs=5, batch_size=1024, lr=1e-4, lam=0.1):
    """
    Train the SOAR model using residual vectors computed as data minus cluster centers.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SOARModel(input_dim, rep_dim, num_reps=num_reps).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    # Convert data to tensors and compute residuals
    data_t = torch.tensor(data, dtype=torch.float32, device=device)
    centers_t = torch.tensor(centers, dtype=torch.float32, device=device)
    labels_t = torch.tensor(labels, dtype=torch.long, device=device)
    residuals = data_t - centers_t[labels_t]

    n_samples = residuals.shape[0]
    for epoch in range(epochs):
        perm = torch.randperm(n_samples, device=device)
        epoch_loss = 0.0
        for i in range(0, n_samples, batch_size):
            idx = perm[i: i + batch_size]
            batch = residuals[idx]
            optimizer.zero_grad()
            combined_rep, reps = model(batch)
            loss_val = soar_loss(batch, combined_rep, reps, lam=lam)
            loss_val.backward()
            optimizer.step()
            epoch_loss += loss_val.item() * batch.size(0)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss/n_samples:.4f}")
    return model


# Section 6: FAISS Index Building and Querying Functions


In [89]:
def build_faiss_index(reps, batch_size=10000, efSearch_val =50):
    """
    Build a FAISS HNSW index on the provided representations with a specified efSearch value.
    """
    rep_dim = reps.shape[1]
    index = faiss.IndexHNSWFlat(rep_dim, 128)
    index.hnsw.efConstruction = 50
    index.hnsw.efSearch = efSearch_val

    num_batches = (reps.shape[0] + batch_size - 1) // batch_size
    for i in range(num_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, reps.shape[0])
        index.add(reps[start:end])
    return index

def dynamic_rerank_count(query_vector, faiss_index, max_candidates=500, base_min=50):
    """
    Determine the candidate pool size for dynamic re-ranking based on query distance distribution.
    """
    distances, candidate_indices = faiss_index.search(query_vector, max_candidates)
    distances = distances[0]
    k = 10
    if k < len(distances) - 1:
        gap = distances[k] - distances[k - 1]
    else:
        gap = 0

    std_topk = np.std(distances[:k])
    dynamic_count = base_min
    if std_topk < 0.01:
        dynamic_count = min(max_candidates, base_min + int((0.01 - std_topk) * 1000))
    dynamic_count = max(dynamic_count, k)
    return dynamic_count, candidate_indices[0][:dynamic_count]

def evaluate_recall(faiss_index, base_data, test_data, true_neighbors, k=10, max_rerank=500):
    """
    Evaluate the recall@k for the FAISS index using dynamic re-ranking.
    """
    n_queries = test_data.shape[0]
    total_recall = 0.0
    for i in range(n_queries):
        query_vec = test_data[i:i+1]
        dynamic_count, dynamic_candidates = dynamic_rerank_count(query_vec, faiss_index, max_candidates=max_rerank)
        distances = np.linalg.norm(query_vec - base_data[dynamic_candidates], axis=1)
        retrieved_indices = dynamic_candidates[np.argsort(distances)][:k]
        gt = set(true_neighbors[i, :k])
        overlap = len(set(retrieved_indices) & gt)
        total_recall += overlap / k
    recall = total_recall / n_queries
    print(f"Avg Recall@{k}: {recall:.4f}")
    return recall



# Section 7: Hyperparameter Tuning Functions


In [90]:
def hyperparam_search_clustering(train_data, test_data, true_neighbors, cluster_options=[128, 256, 512], device=None):
    best_recall = 0
    best_cluster_count = None
    for n_clusters in cluster_options:
        labels, centers = partition_data(train_data, n_clusters=n_clusters)
        model = train_soar_model(train_data, labels, centers, input_dim=train_data.shape[1], rep_dim=train_data.shape[1], num_reps = 6, epochs=5, batch_size=1024)
        train_data_tensor = torch.tensor(train_data, dtype=torch.float32, device=device)
        reps_train, _ = model(train_data_tensor)
        reps_train = reps_train.cpu().detach().numpy()
        index = build_faiss_index(reps_train)
        current_recall = evaluate_recall(index, train_data, test_data, true_neighbors, k=10)
        print(f"Clusters={n_clusters}, Recall@10={current_recall:.4f}")
        if current_recall > best_recall:
            best_recall = current_recall
            best_cluster_count = n_clusters
    print(f"Best cluster count: {best_cluster_count} with recall {best_recall:.4f}")
    return best_cluster_count

def tune_orthogonality(train_data, labels, centers, test_data, true_neighbors, lambdas=[0.01, 0.05, 0.1, 0.001], input_dim=64, rep_dim=64, num_reps=6, epochs=5, batch_size=1024, lr=1e-4):
    best_lambda = None
    best_recall = 0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for lam in lambdas:
        model = train_soar_model(train_data, labels, centers, input_dim, rep_dim, num_reps, epochs, batch_size, lr, lam=lam)
        train_data_tensor = torch.tensor(train_data, dtype=torch.float32, device=device)
        reps_train, _ = model(train_data_tensor)
        reps_train = reps_train.cpu().detach().numpy()
        index = build_faiss_index(reps_train)
        current_recall = evaluate_recall(index, train_data, test_data, true_neighbors, k=10)
        print(f"Lambda={lam}, Recall@10={current_recall:.4f}")
        if current_recall > best_recall:
            best_recall = current_recall
            best_lambda = lam
    print(f"Best lambda: {best_lambda} with recall {best_recall:.4f}")
    return best_lambda, best_recall



# Section 8: Main Pipeline


In [91]:
if __name__ == "__main__":
    h5_path = "sift-128-euclidean.hdf5"

    print("Loading and normalizing data...")
    train_data = load_and_normalize_data_hdf5(h5_path, 'train')
    test_data = load_and_normalize_data_hdf5(h5_path, 'test')
    with h5py.File(h5_path, 'r') as f:
        true_neighbors = np.array(f['neighbors'])

    print("Applying PCA transformation...")
    train_data, test_data = apply_pca(train_data, test_data, n_components=128)

    print("Partitioning training data via clustering...")
    best_cluster_count = 128  #got from parameter testing
    best_lambda = 0.001

    labels, centers = partition_data(train_data, n_clusters=best_cluster_count)
    print("Training the SOAR model...")
    model = train_soar_model(
        train_data, labels, centers,
        input_dim=train_data.shape[1], rep_dim=128, lam=best_lambda, epochs=5
    )

    print("Generating training representations using the SOAR model...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_data_tensor = torch.tensor(train_data, dtype=torch.float32, device=device)
    reps_train, _ = model(train_data_tensor)
    reps_train = reps_train.cpu().detach().numpy()

    print("Generating test representations using the SOAR model...")
    test_data_tensor = torch.tensor(test_data, dtype=torch.float32, device=device)
    reps_test, _ = model(test_data_tensor)
    reps_test = reps_test.cpu().detach().numpy()

    # Testing the effect of different efSearch values
    efSearch_values = [50, 100, 200]
    results = {}
    for ef in efSearch_values:
        print(f"\nBuilding FAISS index with efSearch = {ef}...")
        index = build_faiss_index(reps_train, efSearch_val=ef)
        print("Evaluating Recall@10 on test data...")
        start_time = time.time()
        recall = evaluate_recall(index, reps_train, reps_test, true_neighbors, k=10)
        elapsed_time = time.time() - start_time
        print(f"efSearch = {ef} -- Recall@10: {recall:.4f}, Evaluation time: {elapsed_time:.4f} seconds")
        results[ef] = {'Recall@10': recall, 'EvalTime(s)': elapsed_time}

    print("\nSummary of efSearch evaluations:")
    for ef, metrics in results.items():
        print(f"efSearch = {ef}: Recall@10 = {metrics['Recall@10']:.4f}, Evaluation Time = {metrics['EvalTime(s)']:.4f} seconds")


Loading and normalizing data...
Applying PCA transformation...
Partitioning training data via clustering...
Training the SOAR model...
Epoch 1/5 - Loss: 0.0082
Epoch 2/5 - Loss: 0.0022
Epoch 3/5 - Loss: 0.0019
Epoch 4/5 - Loss: 0.0019
Epoch 5/5 - Loss: 0.0019
Generating training representations using the SOAR model...
Generating test representations using the SOAR model...

Building FAISS index with efSearch = 50...
Evaluating Recall@10 on test data...
Avg Recall@10: 0.9467
efSearch = 50 -- Recall@10: 0.9467, Evaluation time: 5.5634 seconds

Building FAISS index with efSearch = 100...
Evaluating Recall@10 on test data...
Avg Recall@10: 0.9522
efSearch = 100 -- Recall@10: 0.9522, Evaluation time: 7.6918 seconds

Building FAISS index with efSearch = 200...
Evaluating Recall@10 on test data...
Avg Recall@10: 0.9536
efSearch = 200 -- Recall@10: 0.9536, Evaluation time: 11.1224 seconds

Summary of efSearch evaluations:
efSearch = 50: Recall@10 = 0.9467, Evaluation Time = 5.5634 seconds
efSe

#Section 9 Parameter Testing

In [92]:
print("Running hyperparameter search for clustering...")
best_cluster_count = hyperparam_search_clustering(train_data, test_data, true_neighbors,
                                                  cluster_options=[128, 256, 512], device=device)
print(f"Optimal cluster count: {best_cluster_count}")

#Hyperparameter tuning for orthogonality strength (lambda)
print("Tuning orthogonality strength (lambda)...")
best_lambda, best_recall = tune_orthogonality(train_data, labels, centers,
                                              test_data, true_neighbors,
                                              lambdas=[0.01, 0.05, 0.001],
                                              input_dim=train_data.shape[1],
                                              rep_dim=train_data.shape[1],
                                              num_reps=6, epochs=5,
                                              batch_size=1024, lr=1e-4)
print(f"Optimal lambda: {best_lambda} with Recall@10: {best_recall:.4f}")

Running hyperparameter search for clustering...
Epoch 1/5 - Loss: 0.0626
Epoch 2/5 - Loss: 0.0493
Epoch 3/5 - Loss: 0.0469
Epoch 4/5 - Loss: 0.0462
Epoch 5/5 - Loss: 0.0458
Avg Recall@10: 0.6202
Clusters=128, Recall@10=0.6202
Epoch 1/5 - Loss: 0.0614
Epoch 2/5 - Loss: 0.0492
Epoch 3/5 - Loss: 0.0470
Epoch 4/5 - Loss: 0.0463
Epoch 5/5 - Loss: 0.0461
Avg Recall@10: 0.6274
Clusters=256, Recall@10=0.6274
Epoch 1/5 - Loss: 0.0610
Epoch 2/5 - Loss: 0.0492
Epoch 3/5 - Loss: 0.0471
Epoch 4/5 - Loss: 0.0465
Epoch 5/5 - Loss: 0.0463
Avg Recall@10: 0.6207
Clusters=512, Recall@10=0.6207
Best cluster count: 256 with recall 0.6274
Optimal cluster count: 256
Tuning orthogonality strength (lambda)...
Epoch 1/5 - Loss: 0.0329
Epoch 2/5 - Loss: 0.0276
Epoch 3/5 - Loss: 0.0268
Epoch 4/5 - Loss: 0.0267
Epoch 5/5 - Loss: 0.0267
Avg Recall@10: 0.3949
Lambda=0.01, Recall@10=0.3949
Epoch 1/5 - Loss: 0.0542
Epoch 2/5 - Loss: 0.0465
Epoch 3/5 - Loss: 0.0449
Epoch 4/5 - Loss: 0.0440
Epoch 5/5 - Loss: 0.0425
Avg 

In [93]:
pca_dims = [64, 96, 128, None]
results_pca = []

for dim in pca_dims:
    print(f"\n--- PCA = {dim if dim else 'None'} ---")
    if dim is None:
        train_data_pca, test_data_pca = train_data, test_data
    else:
        train_data_pca, test_data_pca = apply_pca(train_data, test_data, n_components=dim)

    labels, centers = partition_data(train_data_pca, n_clusters=256)
    model = train_soar_model(train_data_pca, labels, centers,
                             input_dim=train_data_pca.shape[1],
                             rep_dim=train_data_pca.shape[1],
                             lam=0.05)

    reps_train, _ = model(torch.tensor(train_data_pca, dtype=torch.float32, device=device))
    reps_test, _ = model(torch.tensor(test_data_pca, dtype=torch.float32, device=device))

    reps_train = reps_train.cpu().detach().numpy()
    reps_test = reps_test.cpu().detach().numpy()

    index = build_faiss_index(reps_train)
    start = time.time()
    recall = evaluate_recall(index, reps_train, reps_test, true_neighbors)
    query_time = time.time() - start

    results_pca.append((dim if dim else 'None', recall, query_time))
    print(f"PCA {dim}D → Recall@10: {recall:.4f}, Query Time: {query_time:.2f}s")



--- PCA = 64 ---
Epoch 1/5 - Loss: 0.0665
Epoch 2/5 - Loss: 0.0601
Epoch 3/5 - Loss: 0.0503
Epoch 4/5 - Loss: 0.0429
Epoch 5/5 - Loss: 0.0413
Avg Recall@10: 0.5529
PCA 64D → Recall@10: 0.5529, Query Time: 4.78s

--- PCA = 96 ---
Epoch 1/5 - Loss: 0.0457
Epoch 2/5 - Loss: 0.0407
Epoch 3/5 - Loss: 0.0369
Epoch 4/5 - Loss: 0.0295
Epoch 5/5 - Loss: 0.0270
Avg Recall@10: 0.5866
PCA 96D → Recall@10: 0.5866, Query Time: 4.90s

--- PCA = 128 ---
Epoch 1/5 - Loss: 0.0341
Epoch 2/5 - Loss: 0.0309
Epoch 3/5 - Loss: 0.0293
Epoch 4/5 - Loss: 0.0261
Epoch 5/5 - Loss: 0.0225
Avg Recall@10: 0.5666
PCA 128D → Recall@10: 0.5666, Query Time: 5.11s

--- PCA = None ---
Epoch 1/5 - Loss: 0.0348
Epoch 2/5 - Loss: 0.0308
Epoch 3/5 - Loss: 0.0292
Epoch 4/5 - Loss: 0.0263
Epoch 5/5 - Loss: 0.0210
Avg Recall@10: 0.5599
PCA NoneD → Recall@10: 0.5599, Query Time: 5.16s
